In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# ベースURL（1ページ目）
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
# ページ番号を追加するフォーマット
url_with_page = base_url + "&page={}"

# ヘッダー設定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# スクレイピング処理
def scrape_all_pages():
    properties = []
    page = 1  # 初期ページ番号

    while True:
        url = url_with_page.format(page) if page > 1 else base_url
        print(f"\n[INFO] アクセス中: {url} (ページ {page})")
        
        try:
            response = requests.get(url, headers=headers)
        except Exception as e:
            print(f"[ERROR] リクエスト失敗: {e}")
            break

        # ステータスコードが200以外の場合は終了
        if response.status_code != 200:
            print(f"[ERROR] ステータスコード: {response.status_code}")
            break

        # HTMLを解析
        soup = BeautifulSoup(response.text, "html.parser")
        
        # SUUMOページ内の賃貸物件リストを取得
        property_list = soup.find_all("div", class_="cassetteitem")
        if not property_list:  # 物件リストが取得できなければ終了
            print("[INFO] 物件リストが見つかりません。このページが最後の可能性があります。")
            break

        print(f"[INFO] ページ {page} から {len(property_list)} 件の物件を取得しました。")

        for property in property_list:
            try:
                name = property.find("div", class_="cassetteitem_content-title").text.strip()
                address = property.find("li", class_="cassetteitem_detail-col1").text.strip()
                stations = [station.text.strip() for station in property.find_all("li", class_="cassetteitem_detail-col2")]
                rent = property.find("span", class_="cassetteitem_price--rent").text.strip()

                # 間取り、専有面積の取得
                td_col3 = property.find("td", class_="detailbox-property-col detailbox-property--col3")
                layout = None
                area = None
                if td_col3:
                    divs = td_col3.find_all("div")
                    if len(divs) > 0:
                        layout = divs[0].text.strip()  # 間取り
                    if len(divs) > 1:
                        area = divs[1].text.strip()  # 専有面積

                # 種別、築年の取得
                td_col4 = property.find("td", class_="detailbox-property-col detailbox-property--col4")
                property_type = None
                built_year = None
                if td_col4:
                    divs = td_col4.find_all("div")
                    if len(divs) > 0:
                        property_type = divs[0].text.strip()  # 種別
                    if len(divs) > 1:
                        built_year = divs[1].text.strip()  # 築年

                # データをリストに追加
                properties.append({
                    "物件名": name,
                    "住所": address,
                    "最寄り駅": stations,
                    "賃料": rent,
                    "間取り": layout,
                    "専有面積": area,
                    "種別": property_type,
                    "築年": built_year
                })
            except AttributeError as e:
                print(f"[WARNING] データ取得失敗: {e}")
                continue

        # 次のページへ
        page += 1
        time.sleep(3)  # サーバー負荷軽減のため待機

    return properties

# スクレイピング開始
print("[INFO] スクレイピングを開始します...")
properties = scrape_all_pages()

# データをCSVに保存
if properties:
    df = pd.DataFrame(properties)
    output_file = "rental_properties_debug.csv"
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"[INFO] データをCSVに保存しました: {output_file}")
    print(f"[INFO] 総物件数: {len(properties)} 件")
else:
    print("[INFO] 物件データが取得できませんでした。")


2025-01-24 09:55:30,243 [DEBUG] Starting new HTTPS connection (1): suumo.jp:443


[INFO] スクレイピングを開始します...

[INFO] アクセス中: https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1 (ページ 1)


2025-01-24 09:55:32,977 [DEBUG] https://suumo.jp:443 "GET /jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1 HTTP/1.1" 200 None


[INFO] ページ 1 から 30 件の物件を取得しました。


2025-01-24 09:55:36,538 [DEBUG] Starting new HTTPS connection (1): suumo.jp:443


...

[INFO] ページ 92 から 6 件の物件を取得しました。


2025-01-24 10:01:54,848 [DEBUG] Starting new HTTPS connection (1): suumo.jp:443



[INFO] アクセス中: https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=93 (ページ 93)


2025-01-24 10:01:55,435 [DEBUG] https://suumo.jp:443 "GET /jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=93 HTTP/1.1" 200 19962


[INFO] 物件リストが見つかりません。このページが最後の可能性があります。
[INFO] データをCSVに保存しました: rental_properties_debug.csv
[INFO] 総物件数: 2736 件
